In [1]:
import pandas as pd
import numpy as np
import os, sys, time
import re

# bioPython
from Bio.Seq import Seq

In [2]:
# load
probe_filename = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/PE_TS/PL70-76_amplifier_pools_bits_pegfish_2lvl.xlsx'
probe_df = pd.read_excel(probe_filename)

In [6]:
#sel_pools = ['PL72_emx1_bit_350', 'PL73_hek3_bit_350', 'PL74_rnf2_bit_350',]
sel_pools = ['PL73_hek3_bit_350', 'PL74_rnf2_bit_350', 'PL72_emx1_bit_350']
readout_copy_nums = [4, 8, ]
T7_promoter = Seq('TAATACGACTCACTATAGGG')
T7_promoter_rc = str(T7_promoter.reverse_complement())
# assemble
pool_2_seqs = {}
#pool_2_seqs = {_n:[] for _n in readout_copy_nums}
readout_site_dict = {'name':[], 'seq':[]}
readout_size = 20
spacer = 'AT'

for sel_pool in sel_pools:
    sel_probe_df = probe_df.loc[probe_df['Pool name']==sel_pool]
    for seq in list(sel_probe_df['Sequence'].values):
        # Parse
        fwd_seq, rev_primer_rc = re.findall('[ATCG]+', seq)
        fwd_primer = fwd_seq[:len(rev_primer_rc)]
        target_seq = fwd_seq[len(rev_primer_rc):]
        
        readout_seq = re.findall('[atcg]+', seq)[0]
        readout_site = readout_seq[:readout_size]
        
        #print(fwd_primer, rev_primer_rc, target_seq, readout_site)
        # append
        readout_site_dict['name'].append(sel_pool.split('_bit_350')[0])
        readout_site_dict['name'].append(readout_site)
        for _n in readout_copy_nums:
            _pool_name = f"{sel_pool.split('_bit_350')[0]}-{_n}_readouts"
            if _pool_name not in pool_2_seqs:
                pool_2_seqs[_pool_name] = []
            
            #_final_seq_list = [fwd_primer] + [readout_site]*min(_n,2) + [target_seq] + [readout_site]*max(_n-2, 0) + [rev_primer_rc]
            _final_seq_list = [fwd_primer] + [readout_site]*_n + [target_seq] + [rev_primer_rc] + [T7_promoter_rc]
            _final_seq = spacer.join(_final_seq_list)
            # append
            pool_2_seqs[_pool_name].append(_final_seq)

## add readout info

In [9]:
readout_folder = r'/lab/solexa_weissman/puzheng/References/Readouts'
readout_reference = os.path.join(readout_folder, 'Readout_summary.xlsx')

# load
readout_table = pd.read_excel(readout_reference)
readout_len = 20

In [10]:
annotated_summary_df = summary_df.copy()
probe_bits_list = []
for _seq in annotated_summary_df['Sequence']:
    #print(_seq)
    _seq_bits = []
    for _i in range(0, len(_seq)-readout_len+1):
        _target = _seq[_i:_i+readout_len].upper()
        if _target in readout_table['Target'].values:
            #print(_target)
            _seq_bits.append(readout_table.loc[readout_table['Target']==_target, 'Name'].values[0])
            
    probe_bits_list.append(_seq_bits)
annotated_summary_df.loc[:,'readout'] = [np.unique(_rds)[0] for _rds in probe_bits_list]

In [11]:
annotated_summary_df

,Pool name,Sequence,readout
0,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgaggcggattgagattcgg...,NDB_634
1,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATcgatggtcgtcctcgtttc...,NDB_332
2,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgtttgcgtgtaatcgactc...,NDB_405
3,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgccgtcgtcacgtgcgagt...,NDB_715
4,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgatgcctcttcgatagatt...,NDB_758
...,...,...,...
121,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATggcactaggataactttag...,NDB_398
122,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgtccatgatacgaggtgat...,NDB_741
123,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgcacgtatgtcccgtccat...,NDB_616
124,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATaagggcgatgtaacggcgc...,NDB_905


In [7]:
# create df
summary_dict = {'Pool name': [], 'Sequence': []}
for _pool_name, _seqs in pool_2_seqs.items():
    summary_dict['Pool name'].extend([_pool_name]*len(_seqs))
    summary_dict['Sequence'].extend(_seqs)
summary_df = pd.DataFrame(summary_dict)

In [8]:
summary_df

,Pool name,Sequence
0,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgaggcggattgagattcgg...
1,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATcgatggtcgtcctcgtttc...
2,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgtttgcgtgtaatcgactc...
3,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgccgtcgtcacgtgcgagt...
4,PL73_hek3-4_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgatgcctcttcgatagatt...
...,...,...
121,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATggcactaggataactttag...
122,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgtccatgatacgaggtgat...
123,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATgcacgtatgtcccgtccat...
124,PL72_emx1-8_readouts,CGCGCGCCTTTGGCGGGAAGTCCTGATaagggcgatgtaacggcgc...
